In [ ]:
import pandas as pd
df = pd.read_csv("spotify_genre_data.csv",delimiter=";")
df = df[df['genre'].str.contains('Error') == False]
df = df[df['genre'].notna()] 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load your CSV file
df = pd.read_csv("spotify_genre_data.csv",delimiter=";")  # Change to your actual file name

# Ensure 'genre' column is a list
df['genre'] = df['genre'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Define labeled genres
positive_genres = {'pop', 'hip hop', 'r&b', 'healing', 'meditation'}
negative_genres = {'rock', 'slow', 'indie', 'rap'}

# Function to categorize each genre
def categorize_genre(genres):
    genres = set(genres)  # Convert list to set for easy comparison
    if genres & positive_genres:
        return 'positive'
    elif genres & negative_genres:
        return 'negative'
    else:
        return 'neutral'  # Neutral categories won't be used for ML

# Apply sentiment classification
df['sentiment'] = df['genre'].apply(categorize_genre)

# Remove neutral rows for training
df_train = df[df['sentiment'] != 'neutral'].copy()

# Convert genre list into text format for ML processing
df_train['genre_text'] = df_train['genre'].apply(lambda x: ' '.join(x))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_train['genre_text'], df_train['sentiment'], test_size=0.2, random_state=42)

# Convert text into numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_vec, y_train)

# Predict sentiments for test set
y_pred = model.predict(X_test_vec)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Predict sentiment for the entire dataset
df['predicted_sentiment'] = model.predict(vectorizer.transform(df['genre'].apply(lambda x: ' '.join(x))))
df.drop(columns=['artist'], inplace=True)
df.drop(columns=['track'], inplace=True)
df.drop(columns=['spotify_track_uri'], inplace=True)
df.drop(columns=['sentiment'], inplace=True)
# Display the first few rows of the updated dataset
display(df)  



Model Accuracy: 1.00


,timestamp,genre,predicted_sentiment
0,2024-11-01T14:30:09Z,"[turkish alternative rock, turkish rock]",negative
1,2024-11-01T14:34:46Z,"[turkish alternative rock, turkish rock]",negative
2,2024-11-01T14:38:33Z,"[turkish alternative rock, turkish rock]",negative
3,2024-11-01T14:45:37Z,"[turkish alternative rock, turkish rock]",negative
4,2024-11-01T14:45:51Z,"[turkish alternative rock, turkish rock]",negative
...,...,...,...
228,2024-11-27T10:59:58Z,"[turkish folk, turkish jazz, turkish rock]",negative
229,2024-11-27T10:59:59Z,"[indietronica, modern alternative rock, modern...",negative
230,2024-11-27T11:00:00Z,"[indie pop, liverpool indie, pov: indie]",positive
231,2024-11-27T11:01:37Z,"[indietronica, modern alternative rock, modern...",negative
